In [2]:
## imports
import pandas as pd
import numpy as np
import re
import requests
import yaml


## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 1. Example 1: no credentials; no wrapper

Site: National Assessment of Education Progress (NAEP)

Documentation: https://www.nationsreportcard.gov/api_documentation.aspx

Base link: https://www.nationsreportcard.gov/DataService/GetAdhocData.aspx 

## 1.1 Query to pull some data

In [4]:
## using their example query of 2011 writing scores separated by gender
## based on here - https://stackoverflow.com/questions/40836749/pythonic-way-of-writing-a-single-line-long-string
## using the ( ) syntax to formulate a long
## string without linebreaks added
example_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011')


example_naep_query


'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011'

In [6]:
## use requests to call the api
naep_resp = requests.get(example_naep_query)
naep_resp
print(type(naep_resp))

#<Response [200]> -->this output means that it succeeded



## get the json contents of the response 
## here, we're assuming valid response
naep_resp_j = naep_resp.json()
naep_resp_j

## with result, turn it into a dataframe
naep_resp_d = pd.DataFrame(naep_resp_j['result'])
naep_resp_d

<Response [200]>

<class 'requests.models.Response'>


{'status': 200,
 'serviceVersion': '3.28.2025.1',
 'dwellTimeMS': '0',
 'avgWebHostCPUTotalLoad': 'N/A',
 'dataHitType': 'FROM_MEMORY',
 'Source': 'B11A',
 'result': [{'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '1',
   'varValueLabel': 'Male',
   'value': 139.099504632971,
   'isStatDisplayable': 1,
   'errorFlag': 0},
  {'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '2',
   'varValueLabel': 'Female',
   'value': 158.567104984955,
   'isStatDisp

,year,sample,yearSampleLabel,Cohort,CohortLabel,stattype,subject,grade,scale,jurisdiction,jurisLabel,variable,variableLabel,varValue,varValueLabel,value,isStatDisplayable,errorFlag
0,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,1,Male,139.099505,1,0
1,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,2,Female,158.567105,1,0


In [10]:
naep_resp_j["result"] #just looking at the result-->see its messy so want to make ti a data frame
pd.DataFrame(naep_resp_j['result'])

[{'year': 2011,
  'sample': 'R3',
  'yearSampleLabel': '2011',
  'Cohort': 2,
  'CohortLabel': 'Grade 8',
  'stattype': 'MN:MN',
  'subject': 'WRI',
  'grade': 8,
  'scale': 'WRIRP',
  'jurisdiction': 'NP',
  'jurisLabel': 'National public',
  'variable': 'GENDER',
  'variableLabel': 'Gender',
  'varValue': '1',
  'varValueLabel': 'Male',
  'value': 139.099504632971,
  'isStatDisplayable': 1,
  'errorFlag': 0},
 {'year': 2011,
  'sample': 'R3',
  'yearSampleLabel': '2011',
  'Cohort': 2,
  'CohortLabel': 'Grade 8',
  'stattype': 'MN:MN',
  'subject': 'WRI',
  'grade': 8,
  'scale': 'WRIRP',
  'jurisdiction': 'NP',
  'jurisLabel': 'National public',
  'variable': 'GENDER',
  'variableLabel': 'Gender',
  'varValue': '2',
  'varValueLabel': 'Female',
  'value': 158.567104984955,
  'isStatDisplayable': 1,
  'errorFlag': 0}]

,year,sample,yearSampleLabel,Cohort,CohortLabel,stattype,subject,grade,scale,jurisdiction,jurisLabel,variable,variableLabel,varValue,varValueLabel,value,isStatDisplayable,errorFlag
0,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,1,Male,139.099505,1,0
1,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,2,Female,158.567105,1,0


## 1.2 What happens if there's an error in our query?

In [12]:
## here's a query that from the documentation we know
## won't work since i modified year to 2025 which doesnt
## exist in the data
wrong_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2025')

wrong_naep_query

'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2025'

In [14]:
## use requests to call the api
naep_wrong_resp = requests.get(wrong_naep_query)
naep_wrong_resp


#<Response [400]> -->this output means that it failed (the database does not have these columns so it can't return)

<Response [400]>

In [ ]:
## in the case of this particular api,
## the call returns some response but
## when we try to extract the json containing
## status or results, we get in an error
#naep_wrong_resp.json() # uncomment to see error

### 1.2.2 More all-purpose way of allowing remainder of calls to run: try, except

In [22]:
## putting it in a try; except as general error catching
try:
    results = naep_wrong_resp.json()['result'] #just looking for a result
    print(results) #adding this in it prings nothing and moves to teh except
except Exception as e: #this activates if it fails, like if there are no results i.e. you get <Response [400]>
    print('Failed to get result from API due to error:')
    print(e) # or just: pass


##except = if something doesn't work, do something else

Failed to get result from API due to error:
Invalid control character at: line 1 column 293 (char 292)


In [24]:
try:
    results = naep_resp.json()['result'] #just looking for a result
    print(results) #here it prints the result bc there is one (the try was successsful), so you don't move to the except
except Exception as e: #this activates if it fails, like if there are no results i.e. you get <Response [400]>
    print('Failed to get result from API due to error:')
    print(e) # or just: pass

[{'year': 2011, 'sample': 'R3', 'yearSampleLabel': '2011', 'Cohort': 2, 'CohortLabel': 'Grade 8', 'stattype': 'MN:MN', 'subject': 'WRI', 'grade': 8, 'scale': 'WRIRP', 'jurisdiction': 'NP', 'jurisLabel': 'National public', 'variable': 'GENDER', 'variableLabel': 'Gender', 'varValue': '1', 'varValueLabel': 'Male', 'value': 139.099504632971, 'isStatDisplayable': 1, 'errorFlag': 0}, {'year': 2011, 'sample': 'R3', 'yearSampleLabel': '2011', 'Cohort': 2, 'CohortLabel': 'Grade 8', 'stattype': 'MN:MN', 'subject': 'WRI', 'grade': 8, 'scale': 'WRIRP', 'jurisdiction': 'NP', 'jurisLabel': 'National public', 'variable': 'GENDER', 'variableLabel': 'Gender', 'varValue': '2', 'varValueLabel': 'Female', 'value': 158.567104984955, 'isStatDisplayable': 1, 'errorFlag': 0}]


### 1.2.3 Can usually also find more targeted way but that varies more across APIs

In [26]:
## if we wanted do more specific error catching,
## see that the status == 400 actually appears here
## so could write if else along those lines
naep_wrong_resp.text
naep_resp.text

if "System.Exception" in naep_wrong_resp.text:
    print("NAEP results not found")

'{"statusCode":400,"result": "System.Exception: The query \'SELECT DISTINCT Framework FROM Cycles WHERE Subject=\'WRI\' AND Cohort=2 AND CONVERT(VARCHAR(10),Year)+Sample IN (\'2025R3\')\' did not return exactly 1 framework. Make sure you can trend the years defined for the given subject and cohort.\r\n   at NRCDataService3.GetAdhocData.GetFramework(NDEContext& ndeContext, String subjectCode, List`1 yearSamples, String cohort) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 2547\r\n   at NRCDataService3.GetAdhocData.PopulateBaseOrchestratorRequest() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 2168\r\n   at NRCDataService3.GetAdhocData.ConstructRequest_Datapoint() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 807\r\n   at NRCDataService3.GetAdhocData.Page_Load(Object sender, EventArgs e) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 261"}'

'{"status":200,"serviceVersion":"3.28.2025.1","dwellTimeMS":"0","avgWebHostCPUTotalLoad":"N/A","dataHitType":"FROM_MEMORY","Source":"B11A","result": [{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","jurisLabel":"National public","variable":"GENDER","variableLabel":"Gender","varValue":"1","varValueLabel":"Male","value":139.099504632971,"isStatDisplayable":1,"errorFlag":0},{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","jurisLabel":"National public","variable":"GENDER","variableLabel":"Gender","varValue":"2","varValueLabel":"Female","value":158.567104984955,"isStatDisplayable":1,"errorFlag":0}]}'

NAEP results not found


## Activity 1: writing a function to make multiple, sequential calls

- Say we want to pull the data for grades 4, 8, and 12
- How can we write a function that iterates over a list of those grades and pulls the data for each grade?

**Note**: an ideal function would have arguments for each parameter in the API like subject, subscale, etc. Here we can leave those other parts constant

In [78]:
# your code here


## f string wiht brackets for something to supstitute is a good way to check in strings

def naep_query(grade_list):
    grade_data_list = []
    for grade in grade_list:

        ## Does the seuqence ofr your query matter--> NO (subject adn grade can be diffent)
        ## so if all you're changing is grade, you can put it first
        
        query = (
            f'https://www.nationsreportcard.gov/'
            f'Dataservice/GetAdhocData.aspx?'
            f'type=data&subject=writing&grade={grade}&'
            f'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011')
        
    
        resp = requests.get(query)
        #print(type(resp))

        try:
            data = resp.json()
            grade_data_list.append(data)
        except ValueError as e:
            print(f"JSON decode failed for grade {grade}: {e}") #bc nothing, trying to do the json decode with it wont work, like shouldn;t be doing json on something that faield or doesn't exist



    return pd.DataFrame(grade_data_list)
        

naep_query([4,8,12])
    

JSON decode failed for grade 4: Invalid control character at: line 1 column 293 (char 292)


,status,serviceVersion,dwellTimeMS,avgWebHostCPUTotalLoad,dataHitType,Source,result
0,200,3.28.2025.1,0,N/A,FROM_MEMORY,B11B,"[{'year': 2011, 'sample': 'R3', 'yearSampleLab..."
1,200,3.28.2025.1,0,N/A,FROM_MEMORY,B11B,"[{'year': 2011, 'sample': 'R3', 'yearSampleLab..."


# 2. Example 2: needs credentials; no wrapper

Create an account here: https://www.yelp.com/developers/v3/manage_app

In [84]:
## get the key
API_KEY = "PHsqFjtttEVNAzU5q72xHTNDGu6b8OxuW0rRujRb6aPK0ZNdOKD0hcRyZC52Vadgc1tnTU_v29xzBqjzV4ziOSqq9c8lcVeugsEfiL_qoj0kpoCIiJ8BFkTgyckcaHYx"

In [86]:
## use documentation to define what to search
## doc: https://www.yelp.com/developers/documentation/v3/business_search
## write the query 
base_url = "https://api.yelp.com/v3/businesses/search?"
my_name = "restaurants"
my_location = "Hanover,NH,03755"
yelp_genquery = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location)

## use requests to call the API; here, we're
## passing it our credentials (structure varies
## by API and telling it to only return 10 results
## (max is 50 at once)
header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp = requests.get(yelp_genquery, headers = header)
yelp_genresp

## then, look at structure of response
yelp_genjson = yelp_genresp.json()


<Response [200]>

In [88]:
## example business
yelp_genjson['businesses'][0]

## more automatic way of summarizing but things end up in lists
## within columns for things like categories
yelp_gendf = pd.DataFrame(yelp_genjson['businesses'])
yelp_gendf.head()

{'id': 'wyV_NfYn4ZOfp_sHMDPcAw',
 'alias': 'bistro-at-six-hanover',
 'name': 'Bistro at Six',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/i4jvssUsxa79VYVLOD3TmQ/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/bistro-at-six-hanover?adjust_creative=mz5pBVQ5EzkpqMCzGJlicw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=mz5pBVQ5EzkpqMCzGJlicw',
 'review_count': 2,
 'categories': [{'alias': 'lounges', 'title': 'Lounges'},
  {'alias': 'cocktailbars', 'title': 'Cocktail Bars'},
  {'alias': 'tradamerican', 'title': 'American'}],
 'rating': 4.0,
 'coordinates': {'latitude': 43.7001146, 'longitude': -72.2877078},
 'transactions': [],
 'price': '$$',
 'location': {'address1': '6 E South St',
  'address2': None,
  'address3': '',
  'city': 'Hanover',
  'zip_code': '03755',
  'country': 'US',
  'state': 'NH',
  'display_address': ['6 E South St', 'Hanover, NH 03755']},
 'phone': '+16036430600',
 'display_phone': '(603) 643-0600',
 'distance': 198.6517

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance,business_hours,attributes
0,wyV_NfYn4ZOfp_sHMDPcAw,bistro-at-six-hanover,Bistro at Six,https://s3-media2.fl.yelpcdn.com/bphoto/i4jvss...,False,https://www.yelp.com/biz/bistro-at-six-hanover...,2,"[{'alias': 'lounges', 'title': 'Lounges'}, {'a...",4.0,"{'latitude': 43.7001146, 'longitude': -72.2877...",[],$$,"{'address1': '6 E South St', 'address2': None,...",+16036430600,(603) 643-0600,198.651788,"[{'open': [{'is_overnight': True, 'start': '00...","{'business_temp_closed': None, 'menu_url': Non..."
1,XVGEEIH5rVB2QzW-qywcJw,base-camp-cafe-hanover,Base Camp Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/p8_YiE...,False,https://www.yelp.com/biz/base-camp-cafe-hanove...,256,"[{'alias': 'himalayan', 'title': 'Himalayan/Ne...",4.4,"{'latitude': 43.700626, 'longitude': -72.2887803}",[delivery],$$,"{'address1': '3 Lebanon St', 'address2': 'Ste ...",+16036432007,(603) 643-2007,196.139758,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': 'ht..."
2,8ybF6YyRldtZmU9jil4xlg,mollys-restaurant-and-bar-hanover,Molly's Restaurant & Bar,https://s3-media4.fl.yelpcdn.com/bphoto/TJLrrA...,False,https://www.yelp.com/biz/mollys-restaurant-and...,561,"[{'alias': 'tradamerican', 'title': 'American'...",3.9,"{'latitude': 43.701144, 'longitude': -72.2894249}",[delivery],$$,"{'address1': '43 South Main St', 'address2': '...",+16036432570,(603) 643-2570,250.830160,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': 'ht..."
3,KA8yhrd-ClVYMyOefXdVYg,lous-restaurant-and-bakery-hanover,Lou's Restaurant & Bakery,https://s3-media3.fl.yelpcdn.com/bphoto/VAx8H9...,False,https://www.yelp.com/biz/lous-restaurant-and-b...,430,"[{'alias': 'tradamerican', 'title': 'American'...",4.2,"{'latitude': 43.70143, 'longitude': -72.289001}",[delivery],$$,"{'address1': '30 S Main St', 'address2': '', '...",+16036433321,(603) 643-3321,245.079232,"[{'open': [{'is_overnight': False, 'start': '0...","{'business_temp_closed': None, 'menu_url': 'ht..."
4,5WW4g_LRwau29KyjZGLyAA,sawtooth-kitchen-hanover,Sawtooth Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/61MNG4...,False,https://www.yelp.com/biz/sawtooth-kitchen-hano...,30,"[{'alias': 'chickenshop', 'title': 'Chicken Sh...",4.2,"{'latitude': 43.70158, 'longitude': -72.289641}",[],NaN,"{'address1': '33 S Main St', 'address2': '', '...",+16036435134,(603) 643-5134,242.607552,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': 'ht..."


In [90]:
## more data-specific way of summarizing
## we're doing a simple approach and just retaining
## cols that have a simple str structure
## if doing for real, would want to extract things
def clean_yelp_json(one_biz):

    ## restrict to str cols
    d_str = {key:value for key, value in one_biz.items()
             if type(value) == str}
    
    df_str = pd.DataFrame(d_str, index = [d_str['id']])
    return(df_str)

yelp_stronly = [clean_yelp_json(one_b) for one_b in yelp_genjson['businesses']]
yelp_stronly_df = pd.concat(yelp_stronly)

yelp_stronly_df.head(7)


,id,alias,name,image_url,url,price,phone,display_phone
wyV_NfYn4ZOfp_sHMDPcAw,wyV_NfYn4ZOfp_sHMDPcAw,bistro-at-six-hanover,Bistro at Six,https://s3-media2.fl.yelpcdn.com/bphoto/i4jvss...,https://www.yelp.com/biz/bistro-at-six-hanover...,$$,+16036430600,(603) 643-0600
XVGEEIH5rVB2QzW-qywcJw,XVGEEIH5rVB2QzW-qywcJw,base-camp-cafe-hanover,Base Camp Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/p8_YiE...,https://www.yelp.com/biz/base-camp-cafe-hanove...,$$,+16036432007,(603) 643-2007
8ybF6YyRldtZmU9jil4xlg,8ybF6YyRldtZmU9jil4xlg,mollys-restaurant-and-bar-hanover,Molly's Restaurant & Bar,https://s3-media4.fl.yelpcdn.com/bphoto/TJLrrA...,https://www.yelp.com/biz/mollys-restaurant-and...,$$,+16036432570,(603) 643-2570
KA8yhrd-ClVYMyOefXdVYg,KA8yhrd-ClVYMyOefXdVYg,lous-restaurant-and-bakery-hanover,Lou's Restaurant & Bakery,https://s3-media3.fl.yelpcdn.com/bphoto/VAx8H9...,https://www.yelp.com/biz/lous-restaurant-and-b...,$$,+16036433321,(603) 643-3321
5WW4g_LRwau29KyjZGLyAA,5WW4g_LRwau29KyjZGLyAA,sawtooth-kitchen-hanover,Sawtooth Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/61MNG4...,https://www.yelp.com/biz/sawtooth-kitchen-hano...,NaN,+16036435134,(603) 643-5134
neBEWvgHNhsXIBImCYx_6A,neBEWvgHNhsXIBImCYx_6A,murphy-s-on-the-green-hanover,Murphy’s on the Green,https://s3-media2.fl.yelpcdn.com/bphoto/76DCQr...,https://www.yelp.com/biz/murphy-s-on-the-green...,$$,+16036434075,(603) 643-4075
vMyN7JL5cJExJORgIobbQg,vMyN7JL5cJExJORgIobbQg,tuk-tuk-thai-cuisine-hanover,Tuk Tuk Thai Cuisine,https://s3-media4.fl.yelpcdn.com/bphoto/--bUG3...,https://www.yelp.com/biz/tuk-tuk-thai-cuisine-...,$$,+16032779192,(603) 277-9192


# Activity 2: pull restaurants in a different location

- Try running a business search query for your hometown or another place by constructing a query similar to `yelp_genquery` but changing the location parameter
- Other endpoints require feeding what's called the business' fusion id into the API. Take an id from `yelp_stronly.id` and use the documentation here to pull the reviews for that business: https://docs.developer.yelp.com/reference/v3_business_reviews
- **Challenge**: generalize the previous step by writing a function that (1) takes a list of business ids as an input, (2) calls the reviews API for each id, (3) returns the results, and (4) rowbinds all results, i.e. turns them into a single, usable DataFrame

In [120]:
# your code here
base_url = "https://api.yelp.com/v3/businesses/search?"

my_name = "restaurants"
my_location = "Mountain Lakes,NJ,07046"
yelp_genquery = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location)

header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp = requests.get(yelp_genquery, headers = header)


def clean_yelp_json(one_biz):

    ## restrict to str cols
    d_str = {key:value for key, value in one_biz.items()
             if type(value) == str} 
    
    df_str = pd.DataFrame(d_str, index = [d_str['id']])
    return(df_str)

yelp_stronly = [clean_yelp_json(one_b) for one_b in yelp_genjson['businesses']]
yelp_stronly_df = pd.concat(yelp_stronly)

yelp_stronly_df.head(7)


,id,alias,name,image_url,url,price,phone,display_phone
-K5G2Rfv10HKWohSd-JlNg,-K5G2Rfv10HKWohSd-JlNg,hapgoods-mountain-lakes,Hapgoods,https://s3-media1.fl.yelpcdn.com/bphoto/eyoWNh...,https://www.yelp.com/biz/hapgoods-mountain-lak...,$$,+19732993399,(973) 299-3399
4OSsfDyyD8MMeQzcOG0TVg,4OSsfDyyD8MMeQzcOG0TVg,station-restaurant-at-mountain-mountain-lakes,Station Restaurant At Mountain,https://s3-media2.fl.yelpcdn.com/bphoto/GMsNkN...,https://www.yelp.com/biz/station-restaurant-at...,$$$,+19733355330,(973) 335-5330
_-vnkIGEQ3ZbFGgX1UkKWA,_-vnkIGEQ3ZbFGgX1UkKWA,black-rabbit-saloon-montville,Black Rabbit Saloon,https://s3-media1.fl.yelpcdn.com/bphoto/tiNPnh...,https://www.yelp.com/biz/black-rabbit-saloon-m...,$$,+19735883340,(973) 588-3340
hhp3R7EJEe7dAw_SnrkWEQ,hhp3R7EJEe7dAw_SnrkWEQ,barka-mountain-lakes,Barka,https://s3-media4.fl.yelpcdn.com/bphoto/qZ3WgE...,https://www.yelp.com/biz/barka-mountain-lakes?...,$$$,+19739173386,(973) 917-3386
eeoHqNZqned1HHctvoesIA,eeoHqNZqned1HHctvoesIA,thonglor-thai-bistro-denville-2,Thonglor Thai Bistro,https://s3-media1.fl.yelpcdn.com/bphoto/hfO78u...,https://www.yelp.com/biz/thonglor-thai-bistro-...,$$,+19734536991,(973) 453-6991
AV-I_BG0QmOtR17VFjDcIw,AV-I_BG0QmOtR17VFjDcIw,sazon-latino-restaurant-boonton,Sazon Latino Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/sAzcGF...,https://www.yelp.com/biz/sazon-latino-restaura...,$,+19737944121,(973) 794-4121
hT75huok1_mLXjtOF2Fpdw,hT75huok1_mLXjtOF2Fpdw,zalim-hot-chicken-and-burgers-parsippany,Zalim Hot Chicken & Burgers,https://s3-media2.fl.yelpcdn.com/bphoto/3yzeWH...,https://www.yelp.com/biz/zalim-hot-chicken-and...,NaN,,


In [130]:
## Other endpoints require feeding what's called the business' fusion id into the API. 
## Take an id from yelp_stronly.id and use the documentation here to pull the reviews for that business:
## https://docs.developer.yelp.com/reference/v3_business_reviews

yelp_stronly_df.id.iloc[0]


url = "https://api.yelp.com/v3/businesses/-K5G2Rfv10HKWohSd-JlNg/reviews?"  # basically just slotting in the id inbetween business and reviews
response = requests.get(url, headers=header)
print(response.text)


biz_id = yelp_stronly_df.id.iloc[0]

# Correctly construct the URL
url = f"https://api.yelp.com/v3/businesses/{biz_id}/reviews"


response = requests.get(url, headers=headers)

# Check the status
if response.status_code == 200:
    reviews = response.json()["reviews"]
    reviews_df = pd.DataFrame(reviews)
    print(reviews_df[["user", "rating", "text", "time_created"]])
else:
    print(f"Error {response.status_code}: {response.json()}")


'-K5G2Rfv10HKWohSd-JlNg'

{"error": {"code": "NOT_FOUND", "description": "Resource could not be found."}}


NameError: name 'headers' is not defined